# Model Evaluation
Using this notebook to evaluate all of the models

In [1]:
# model training using tensorflow
import numpy as np
import os
from matplotlib import pyplot

import pandas as pd

import tensorflow as tf

from tensorflow import keras

In [2]:
'''
Configuration Code
'''
def get_n_cores():
    """The NSLOTS variable, If NSLOTS is not defined throw an exception."""
    nslots = os.getenv("NSLOTS")
    if nslots is not None:
        return int(nslots)
    raise ValueError("Environment variable NSLOTS is not defined.")


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

NUM_GPUS = len(tf.config.experimental.list_physical_devices("GPU"))
print("Num GPUs Available: ", NUM_GPUS)
if NUM_GPUS > 0:
    print(os.getenv("CUDA_VISIBLE_DEVICES"))

tf.config.set_soft_device_placement(True)
tf.keras.backend.set_floatx("float32")
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(get_n_cores())

Num GPUs Available:  1
0


In [3]:
# pull the filenames of all experiments
models = [m for m in next(os.walk("models"))[1] if "experiment" in m]
models = sorted(models)
models

['experiment_001',
 'experiment_002',
 'experiment_003',
 'experiment_004',
 'experiment_004r',
 'experiment_005a',
 'experiment_005b',
 'experiment_005ra',
 'experiment_005rb']

In [4]:
'''
Try the test set on each model and gather data in list of dictionaries
'''

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

TEST_DIRECTORY = os.path.join(os.getcwd(), 'test_data')

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_DIRECTORY,
    seed=123,
    image_size=(224,224),
    shuffle=False
)

results = []

for m in models:
    model = keras.models.load_model("models/{}".format(m))
    
    predictions = np.argmax(model.predict(test_ds), axis=1)
    actual = np.concatenate([y for x, y in test_ds], axis=0)
    
    report = classification_report(actual, predictions, output_dict=True)
    cmatrix = confusion_matrix(actual, predictions)
    
    results.append({
        "name": m,
        "size": os.path.getsize("models/{}/saved_model.pb".format(m)),
        "layers": len(model.layers),
        "accuracy": report['accuracy'],
        "true_pos": cmatrix[1,1],
        "true_neg": cmatrix[0,0],
        "false_pos": cmatrix[0,1],
        "false_neg": cmatrix[1,0],
        "pos_recall": report['1']['recall'],
        "neg_recall": report['0']['recall'],
        "pos_precision": report['1']['precision'],
        "neg_precision": report['0']['precision'],
        "pos_f1-score": report['1']['f1-score'],
        "neg_f1-score": report['0']['f1-score'],
    })

Found 300 files belonging to 2 classes.


In [11]:
data = pd.DataFrame(results)
data = data.set_index("name")
data

,size,layers,accuracy,true_pos,true_neg,false_pos,false_neg,pos_recall,neg_recall,pos_precision,neg_precision,pos_f1-score,neg_f1-score
name,,,,,,,,,,,,,
experiment_001,137029,8,0.946667,138,146,4,12,0.920000,0.973333,0.971831,0.924051,0.945205,0.948052
experiment_002,148884,9,0.946667,140,144,6,10,0.933333,0.960000,0.958904,0.935065,0.945946,0.947368
experiment_003,177253,11,0.946667,143,141,9,7,0.953333,0.940000,0.940789,0.952703,0.947020,0.946309
experiment_004,483616,12,0.950000,145,140,10,5,0.966667,0.933333,0.935484,0.965517,0.950820,0.949153
experiment_004r,494670,12,0.976667,145,148,2,5,0.966667,0.986667,0.986395,0.967320,0.976431,0.976898
experiment_005a,4024888,7,0.956667,144,143,7,6,0.960000,0.953333,0.953642,0.959732,0.956811,0.956522
experiment_005b,4165246,7,0.986667,149,147,3,1,0.993333,0.980000,0.980263,0.993243,0.986755,0.986577
experiment_005ra,4035345,7,0.936667,141,140,10,9,0.940000,0.933333,0.933775,0.939597,0.936877,0.936455
experiment_005rb,4180621,7,0.996667,149,150,0,1,0.993333,1.000000,1.000000,0.993377,0.996656,0.996678


In [12]:
data.to_csv("results.csv")